In [ ]:
class Nodo():
    def __init__(self, estado, padre, accion):
        self.estado = estado  # Estado es una tupla de tuplas (matriz 3x3)
        self.padre = padre
        self.accion = accion

class Frontera():
    def __init__(self):
        self.frontera =[]

    def empty(self):
        return (len(self.frontera) == 0)

    def add(self, nodo):
        self.frontera.append(nodo)

    def eliminar(self):
        # LIFO o FIFO
        pass

    def contiene_estado(self, estado):
        return any(nodo.estado == estado for nodo in self.frontera)

class Pila(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():
            raise Exception("Frontera vacia")
        else:
            # Guardamos el ultimo item en la lista
            # (el cual es el nodo recientemente añadido)
            nodo = self.frontera[-1]
            # Guardamos todos los items excepto el
            # ultimo (eliminamos)
            self.frontera = self.frontera[:-1]
            return nodo

class Cola(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():
            raise Exception("Frontera vacia")
        else:
            # Guardamos el primer item en la lista
            # (el cual es el nodo añadido de primero)
            nodo = self.frontera[0]
            # Guardamos todos los items excepto el
            # primero (eliminamos)
            self.frontera = self.frontera[1:]
            return nodo

# Clase para el puzzle deslizante 3x3
class PuzzleDeslizante():
    def __init__(self, estado_inicial, estado_objetivo):
        self.inicio = estado_inicial
        self.objetivo = estado_objetivo
        self.solucion = None

    def print(self, estado=None):
        if estado is None:
            estado = self.inicio
        for fila in estado:
            print(' '.join(str(x) if x != 0 else ' ' for x in fila))
        print()

    def encontrar_vacio(self, estado):
        for i in range(3):
            for j in range(3):
                if estado[i][j] == 0:
                    return (i, j)

    def mover(self, estado, direccion):
        i, j = self.encontrar_vacio(estado)
        nuevo_estado = [list(fila) for fila in estado]
        if direccion == "up" and i > 0:
            nuevo_estado[i][j], nuevo_estado[i-1][j] = nuevo_estado[i-1][j], nuevo_estado[i][j]
        elif direccion == "down" and i < 2:
            nuevo_estado[i][j], nuevo_estado[i+1][j] = nuevo_estado[i+1][j], nuevo_estado[i][j]
        elif direccion == "left" and j > 0:
            nuevo_estado[i][j], nuevo_estado[i][j-1] = nuevo_estado[i][j-1], nuevo_estado[i][j]
        elif direccion == "right" and j < 2:
            nuevo_estado[i][j], nuevo_estado[i][j+1] = nuevo_estado[i][j+1], nuevo_estado[i][j]
        else:
            return None
        return tuple(tuple(fila) for fila in nuevo_estado)

    def vecinos(self, estado):
        acciones = ["up", "down", "left", "right"]
        resultado = []
        for accion in acciones:
            nuevo_estado = self.mover(estado, accion)
            if nuevo_estado is not None:
                resultado.append((accion, nuevo_estado))
        return resultado

    def soluble(self, estado):
        plano = sum(estado, ())  # Convierte la matriz en una tupla plana
        inversiones = 0
        lista = [x for x in plano if x != 0]
        for i in range(len(lista)):
            for j in range(i + 1, len(lista)):
                if lista[i] > lista[j]:
                    inversiones += 1
        return inversiones % 2 == 0

    def solve(self):
        if not self.soluble(self.inicio):
            print("Este puzzle no es resoluble.")
            return

        self.num_explorados = 0
        start = Nodo(estado=self.inicio, padre=None, accion=None)
        frontera = Cola()  # Usa Cola para BFS (solución más corta)
        frontera.add(start)
        self.explorado = set()

        while True:
            if frontera.empty():
                raise Exception("No hay solución")
            nodo = frontera.eliminar()
            self.num_explorados += 1

            if nodo.estado == self.objetivo:
                acciones = []
                estados = []
                while nodo.padre is not None:
                    acciones.append(nodo.accion)
                    estados.append(nodo.estado)
                    nodo = nodo.padre
                acciones.reverse()
                estados.reverse()
                self.solucion = (acciones, estados)
                return

            self.explorado.add(nodo.estado)
            for accion, estado in self.vecinos(nodo.estado):
                if not frontera.contiene_estado(estado) and estado not in self.explorado:
                    hijo = Nodo(estado=estado, padre=nodo, accion=accion)
                    frontera.add(hijo)

# Ejemplo de uso:
estado_inicial = (
    (1, 2, 3),
    (8, 0, 4),
    (7, 5, 6)
)

estado_objetivo = (
    (1, 2, 3),
    (4, 5, 6),
    (7, 8, 0)
)

puzzle = PuzzleDeslizante(estado_inicial, estado_objetivo)
print("Puzzle inicial:")
puzzle.print()
print("Resolviendo...")
puzzle.solve()

if puzzle.solucion:
    print("Estados explorados:", puzzle.num_explorados)
    print("Solución (acciones):", puzzle.solucion[0])
    print("Estados intermedios:")
    for estado in puzzle.solucion[1]:
        puzzle.print(estado)

Puzzle inicial:
1 2 3
8   4
7 5 6

Resolviendo...
Estados explorados: 2104
Solución (acciones): ['left', 'down', 'right', 'up', 'right', 'down', 'left', 'left', 'up', 'right', 'down', 'right']
Estados intermedios:
1 2 3
  8 4
7 5 6

1 2 3
7 8 4
  5 6

1 2 3
7 8 4
5   6

1 2 3
7   4
5 8 6

1 2 3
7 4  
5 8 6

1 2 3
7 4 6
5 8  

1 2 3
7 4 6
5   8

1 2 3
7 4 6
  5 8

1 2 3
  4 6
7 5 8

1 2 3
4   6
7 5 8

1 2 3
4 5 6
7   8

1 2 3
4 5 6
7 8  

